In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

In [12]:
def cleanString(s):
    return s.replace('\n', '').replace('\r', '').strip()

In [13]:
def processHTML(soup, index, URL):
    class_name_name = "hero-card-panel-head__name"
    hero_name = soup.find(class_=class_name_name).text
    str_details = []
    str_details.append(str(index))
    str_details.append(cleanString(hero_name))
    
    class_name_details = "heroes_person_details_list"
    hero_details_table_headers = soup.find(class_=class_name_details).find_all('dt')
    
    birth_date = ''
    birth_place = ''
    title = ''
    military_unit = ''
    for table_header in hero_details_table_headers:
        table_header_text = cleanString(table_header.text)
        next_sibling_text = cleanString(table_header.next_sibling.next_sibling.text)
        
        if table_header_text == 'Дата рождения':
            birth_date = next_sibling_text
            
        if table_header_text == 'Место рождения':
            birth_place = next_sibling_text
        
        if table_header_text == 'Воинское звание':
            title = next_sibling_text
            
        if table_header_text == 'Воинская часть':
            military_unit = next_sibling_text
    
    str_details.append(birth_date)
    str_details.append(birth_place)
    str_details.append(title)
    str_details.append(military_unit)
    str_details.append(URL)

    return str_details

In [14]:
def tryGetHero(index):
    URL = "https://pamyat-naroda.ru/heroes/person-hero" + str(index) + "/"

    try:
        # не по всем урлам есть герой. если ответа нет > 0.5s, то переходим к след url
        page = requests.get(URL, timeout=0.5)
    except Exception as e:
        return

    try:
        soup = BeautifulSoup(page.content, "html.parser")
        return processHTML(soup, index, URL)
    except Exception as e:
        return

In [20]:
heroes = []
for i in tqdm(range(10)):
# for i in tqdm(range(1000000000)):
    hero = tryGetHero(i)

    if hero:
        heroes.append(hero)
        
    # time.sleep(???) # в секундах

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.64it/s]


In [ ]:
print(len(heroes))

In [134]:
df = pd.DataFrame(heroes, columns = ['id', 'name', 'birth date', 'birth place', 'title', 'military unit', 'link'])
 
df.to_csv('./heroes.csv', index=False)